In [ ]:
! pip install evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer , PretrainedConfig, set_seed
import evaluate
import numpy as np

In [ ]:


from camel_tools.utils.dediac import dediac_ar, dediac_bw
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.disambig.bert import BERTUnfactoredDisambiguator
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.charsets import UNICODE_PUNCT_CHARSET, AR_CHARSET, AR_LETTERS_CHARSET
from camel_tools.utils.charmap import CharMapper
from camel_tools.utils.transliterate import Transliterator

import pandas as pd
import numpy as np
import datasets
import os
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
from sklearn.metrics import cohen_kappa_score

from transformers import AutoConfig, BertForSequenceClassification, AutoTokenizer, BertTokenizer, PretrainedConfig
from transformers import BertConfig, BertForPreTraining, load_tf_weights_in_bert
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch.nn.functional as F
#import numpy as np
#from transformers import Trainer
import torch
#import sys
#import os
#sys.path.append(os.getcwd())
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from src.loss_functions import *


barec_7_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 2,
6: 2,
7: 2,
8: 3,
9: 3,
10: 4,
11: 4,
12: 5,
13: 5,
14: 6,
15: 6,
16: 7,
17: 7,
18: 7,
19: 7
}

barec_5_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 1,
6: 1,
7: 1,
8: 2,
9: 2,
10: 2,
11: 2,
12: 3,
13: 3,
14: 4,
15: 4,
16: 5,
17: 5,
18: 5,
19: 5
}

barec_3_dict = {
1: 1,
2: 1,
3: 1,
4: 1,
5: 1,
6: 1,
7: 1,
8: 1,
9: 1,
10: 1,
11: 1,
12: 2,
13: 2,
14: 3,
15: 3,
16: 3,
17: 3,
18: 3,
19: 3
}

In [ ]:
loss_type = "CE"
d_mat_type = "default" #don't change
n_levels = 13 #don't change

checkpoint = "aubmindlab/bert-base-arabertv02"

data_path = '/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv' # DATA PATH

input_text = "word_sents"

In [ ]:
# save_dir = "/content/drive/MyDrive/MBZ/Thesis/subsets"+checkpoint.split("/")[1]+"_"+input_text+"_"+loss_type+"_"+d_mat_type+"_"+str(n_levels)+"levels"  # SAVE DIRECTORY
# out_xlsx = "/content/drive/MyDrive/MBZ/Thesis/subsets/"+checkpoint.split("/")[1]+"_"+str(n_levels)+"levels"+".xlsx"



In [ ]:
if d_mat_type == "default":
    d_matrix =  [[abs(i-j) for i in range(n_levels)] for j in range(n_levels)]
else:
    d_matrix = [[(abs(i-j)/18)+(abs(barec_7_dict[i+1]-barec_7_dict[j+1])/6)+(abs(barec_5_dict[i+1]-barec_5_dict[j+1])/4)+(abs(barec_3_dict[i+1]-barec_3_dict[j+1])/2) for i in range(19)] for j in range(19)]


In [ ]:
losses_dict = {"CE": Trainer,}
loss_function = losses_dict[loss_type]


In [ ]:
all_df = pd.read_csv(data_path, header=0)
# all_df = all_df[all_df['RL_num_19']>11]



In [ ]:
level_map = {}  # Initialize an empty dictionary for the mapping

for i in range(1, 20):  # Iterate through values 1 to 19
    if i < 12:
        level_map[i] = i  # Map values less than 12 to i
    else:
        level_map[i] = 12  # Map values 12 and above to 12

all_df['RL_num_13'] = all_df['RL_num_19'].map(level_map)  # Apply the mapping to the 'RL_num_19' column and store the result in a new column 'level_map'

In [ ]:
all_df['RL_num_13'].value_counts()

In [ ]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

real_names = {
    input_text: DATA_COLUMN,
    'RL_num_'+str(n_levels): LABEL_COLUMN
}

all_df.rename(columns= real_names, inplace=True)

In [ ]:
minus_mapper = {}
for i in range(n_levels):
  minus_mapper[i+1] = i
all_df = all_df.replace({LABEL_COLUMN: minus_mapper})

In [ ]:
all_df[LABEL_COLUMN].unique()

array([ 9,  3, 11, 10,  6,  5,  2,  4,  7,  8,  1,  0])

In [ ]:
len(all_df)

In [ ]:

all_df = all_df.groupby('Split')

all_df = all_df[[DATA_COLUMN, LABEL_COLUMN]]
all_df.columns = [DATA_COLUMN, LABEL_COLUMN]

train_df = all_df.get_group('Train')
#train_df = train_df.head(4565)
dev_df = all_df.get_group('Dev')
test_df = all_df.get_group('Test')
#tune_df = all_df.get_group('Tune')



In [ ]:
len(train_df)

In [ ]:
set_seed(42)

In [ ]:
train = datasets.Dataset.from_pandas(train_df)
dev = datasets.Dataset.from_pandas(dev_df)
test = datasets.Dataset.from_pandas(test_df)
#tune = datasets.Dataset.from_pandas(tune_df)
dataset = load_dataset("labr") #dump loading .. only to match the dataset template from huggingface
dataset['train'] = train
#dataset['tune'] = tune
dataset['dev'] = dev
dataset['test'] = test

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.02k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.83M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/919k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11760 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2935 [00:00<?, ? examples/s]

In [ ]:
 len(dataset['train'])

In [ ]:
len(dataset['dev'])

In [ ]:
len(dataset['test'])

In [ ]:
train_size = len(dataset['train'])
dev_size = len(dataset['dev'])
test_size = len(dataset['test'])

# Define subsets of the training data
# train_size // 8, train_size // 6, train_size // 4, [train_size // 2,
subset_sizes =  [train_size ]

In [ ]:
def model_init():
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = n_levels, ignore_mismatched_sizes=True)
  model.dist_matix = [[abs(i-j) for i in range(model.num_labels)] for j in range(model.num_labels)]
  for param in model.parameters(): param.data = param.data.contiguous()
  return model

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds,digits=4))
  print(confusion_matrix(p.label_ids,preds))

  #f1_Positive = f1_score(p.label_ids,preds,pos_label=1,average='binary')
  #f1_Negative = f1_score(p.label_ids,preds,pos_label=0,average='binary')
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  #QWK = cohen_kappa_score(p.label_ids, preds, weights='quadratic')
  return {
      #'f1_pos': f1_Positive,
      #'f1_neg': f1_Negative,
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
      #'Quadratic Weighted Kappa': QWK
  }

In [ ]:
# plus_mapper = {}
# for i in range(19):
#   plus_mapper[i] = i+1

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def rank_simple(vector):
    return [i+1 for i in sorted(range(len(vector)), key=vector.__getitem__, reverse=True)]

In [ ]:
####### BAREC levels to SAMER levels
samer_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1,
    6: 1,
    7: 1,
    8: 2,
    9: 2,
    10: 2,
    11: 2,
    12: 3,
    13: 3,
    14: 4,
    15: 4,
    16: 5,
    17: 5,
    18: 5,
    19: 5
}

###### BAREC levels to BAREC Groups
barec_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 3,
    9: 3,
    10: 4,
    11: 4,
    12: 5,
    13: 5,
    14: 6,
    15: 6,
    16: 6,
    17: 7,
    18: 7,
    19: 7
}


In [ ]:
for subset_size in subset_sizes:
    print('training model with subsize: ', subset_size)
    save_dir = "/content/drive/MyDrive/MBZ/Thesis/bert_13_levels"  # SAVE DIRECTORY
    out_xlsx = "/content/drive/MyDrive/MBZ/Thesis/bert_13_levels/"+str(13)+"levels_"+".xlsx"
    # Create subset of the training data
    train_subset = dataset['train'].select(range(subset_size))
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def tokenize_function(example):
        return tokenizer(example[DATA_COLUMN], truncation=True, max_length=512)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_train_subset = train_subset.map(tokenize_function, batched=True)
    #tokenized_datasets = tokenized_datasets.remove_columns(LABEL_COLUMN)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(save_dir,
                                  evaluation_strategy="epoch",
                                  num_train_epochs=6,
                                  per_device_train_batch_size= 64,
                                  per_device_eval_batch_size=16,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="eval_loss",
                                  greater_is_better=False,
                                  save_strategy="epoch",
                                  #overwrite_output_dir=True,
                                  #save_steps=496,
                                  save_total_limit=1,
                                #   push_to_hub=True,
                                #   hub_token='hub_token',
                                  #learning_rate=lr
                                  )
    trainer = loss_function(model_init=model_init,
                  args = training_args,
                  train_dataset = tokenized_train_subset,
                  eval_dataset = tokenized_datasets['dev'],
                  data_collator=data_collator,
                  tokenizer=tokenizer,
                  compute_metrics = compute_metrics)
    trainer.train()
    trainer.save_model(save_dir)

    preds, labels, metrics = trainer.predict(tokenized_datasets['dev'])

    # probs = {}
    # for i in range(19):
    #   probs[i+1] = []
    # ranks = {}
    # for i in range(19):
    #   ranks[i+1] = []
    texts = []
    original_texts = []
    labels = []
    predictions = []
    # samer_labels = []
    # samer_predictions = []
    # barec_labels = []
    # barec_predictions = []
    # is_equal = []
    # samer_is_equal = []
    # barec_is_equal = []
    # within_one = []
    # within_top_2 = []
    # within_top_3 = []
    # within_top_4 = []
    # rank_of_correct = []
    # diff = []

    for i in range(len(preds)):
      texts.append(list(dev_df['text'])[i])
      # texts.append(list(train_df['text'])[i])
      # original_texts.append(list(original_test['text'])[i])
      labels.append(list(dev_df['label'])[i]+1)
      # labels.append(list(train_df['label'])[i]+1)
      predictions.append(argmax(preds[i])+1)
    #   samer_labels.append(samer_dict[labels[-1]])
    #   barec_labels.append(barec_dict[labels[-1]])
    #   samer_predictions.append(samer_dict[predictions[-1]])
    #   barec_predictions.append(barec_dict[predictions[-1]])
    #   softs = softmax(preds[i])
    #   rank = rank_simple(softs)

    #   if labels[-1] == predictions[-1]:
    #     is_equal.append(1)
    #   else:
    #     is_equal.append(0)

    #   if samer_labels[-1] == samer_predictions[-1]:
    #     samer_is_equal.append(1)
    #   else:
    #     samer_is_equal.append(0)

    #   if barec_labels[-1] == barec_predictions[-1]:
    #     barec_is_equal.append(1)
    #   else:
    #     barec_is_equal.append(0)

    #   if abs(labels[-1] - predictions[-1]) <= 1:
    #     within_one.append(1)
    #   else:
    #     within_one.append(0)

    #   if labels[-1] in rank[:2]:
    #     within_top_2.append(1)
    #   else:
    #     within_top_2.append(0)

    #   if labels[-1] in rank[:3]:
    #     within_top_3.append(1)
    #   else:
    #     within_top_3.append(0)

    #   if labels[-1] in rank[:4]:
    #     within_top_4.append(1)
    #   else:
    #     within_top_4.append(0)

    #   rank_of_correct.append(rank.index(labels[-1])+1)
    #   diff.append(max([labels[-1],predictions[-1]])-min([labels[-1],predictions[-1]]))
    #   for j in range(19):
    #     probs[j+1].append(softs[j])
    #     ranks[j+1].append(rank[j])



    # QWK = cohen_kappa_score(labels, predictions, weights='quadratic')
    # acc = sum(is_equal)/len(is_equal)
    # acc_samer = sum(samer_is_equal)/len(samer_is_equal)
    # acc_barec = sum(barec_is_equal)/len(barec_is_equal)
    # acc_within_one_level = sum(within_one)/len(within_one)
    # acc_top_2 = sum(within_top_2)/len(within_top_2)
    # acc_top_3 = sum(within_top_3)/len(within_top_3)
    # acc_top_4 = sum(within_top_4)/len(within_top_4)
    # avg_rank = sum(rank_of_correct)/len(rank_of_correct)
    # avg_distance = sum(diff)/len(diff)

    # print(f"Accuracy: {acc*100:.4f}")
    # print(f"Accuracy with margin of one level: {acc_within_one_level*100:.4f}")
    # print(f"Accuracy of top 2 choices: {acc_top_2*100:.4f}")
    # print(f"Accuracy of top 3 choices: {acc_top_3*100:.4f}")
    # print(f"Accuracy of top 4 choices: {acc_top_4*100:.4f}")
    # print(f"Accuracy of SAMER levels: {acc_samer*100:.4f}")
    # print(f"Accuracy of BAREC groups: {acc_barec*100:.4f}")
    # print(f"Quadratic Weighted Kappa: {QWK*100:.4f}")
    # print(f"Average rank of correct label: {avg_rank:.4f}")
    # print(f"Average distance between labels and predictions: {avg_distance:.4f}")


    v = {
        #'original text': original_texts,
        'text': texts,
        'label': labels,
        'prediction': predictions,
        # 'is_equal': is_equal,
        # 'samer_label': samer_labels,
        # 'samer_prediction': samer_predictions,
        # 'samer_is_equal': samer_is_equal,
        # 'barec_group_label': barec_labels,
        # 'barec_group_prediction': barec_predictions,
        # 'barec_group_is_equal': barec_is_equal,
        # 'within_one_level': within_one,
        # 'within_top2_ranks': within_top_2,
        # 'within_top3_ranks': within_top_3,
        # 'within_top4_ranks': within_top_4,
        # 'rank_of_correct_label': rank_of_correct,
        # 'diff': diff
    }

    # for i in range(1,20):
    #   s = 'p'+str(i)
    #   v[s] = probs[i]
    # for i in range(1,20):
    #   s = 'rank'+str(i)
    #   v[s] = ranks[i]

    final_df = pd.DataFrame.from_dict(v)
    final_df.to_excel(out_xlsx ,index=False)

training model with subsize:  52521


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nour-rabih (noor-rabie) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy
1,1.141600,0.878652,0.504297,0.493309,0.527172,0.698439
2,0.761300,0.887612,0.500424,0.519447,0.515992,0.693792
3,0.604200,0.936148,0.527351,0.500981,0.565239,0.693554
4,0.380800,1.077960,0.525081,0.512787,0.552325,0.684737
5,0.247400,1.223517,0.518081,0.499096,0.546771,0.685810
6,0.185600,1.343911,0.512149,0.495710,0.537111,0.676278


              precision    recall  f1-score   support

           0     0.3333    0.2903    0.3103        31
           1     0.2623    0.3902    0.3137        41
           2     0.3797    0.6068    0.4671       117
           3     0.4638    0.5079    0.4848        63
           4     0.4198    0.4857    0.4503       280
           5     0.3738    0.4255    0.3980       188
           6     0.5088    0.6111    0.5553       666
           7     0.6428    0.6297    0.6362       740
           8     0.5597    0.5211    0.5397       261
           9     0.6547    0.7482    0.6983      1219
          10     0.4633    0.3033    0.3666       521
          11     0.8578    0.8061    0.8312      4266

    accuracy                         0.6984      8393
   macro avg     0.4933    0.5272    0.5043      8393
weighted avg     0.7080    0.6984    0.7004      8393

[[   9   10    8    0    1    0    1    0    0    1    0    1]
 [   8   16    9    0    1    2    1    0    1    1    0    2]
 [   2 

OSError: [Errno 95] Operation not supported: '/content/drive/MyDrive/MBZ/Thesis/bert_13_levels/Results Template 13 levels.gsheet'

In [ ]:
trainer.save_model(save_dir)

preds, labels, metrics = trainer.predict(tokenized_datasets['dev'])

# probs = {}
# for i in range(19):
#   probs[i+1] = []
# ranks = {}
# for i in range(19):
#   ranks[i+1] = []
texts = []
original_texts = []
labels = []
predictions = []
# samer_labels = []
# samer_predictions = []
# barec_labels = []
# barec_predictions = []
# is_equal = []
# samer_is_equal = []
# barec_is_equal = []
# within_one = []
# within_top_2 = []
# within_top_3 = []
# within_top_4 = []
# rank_of_correct = []
# diff = []

for i in range(len(preds)):
  texts.append(list(dev_df['text'])[i])
  # texts.append(list(train_df['text'])[i])
  # original_texts.append(list(original_test['text'])[i])
  labels.append(list(dev_df['label'])[i]+1)
  # labels.append(list(train_df['label'])[i]+1)
  predictions.append(argmax(preds[i])+1)
#   samer_labels.append(samer_dict[labels[-1]])
#   barec_labels.append(barec_dict[labels[-1]])
#   samer_predictions.append(samer_dict[predictions[-1]])
#   barec_predictions.append(barec_dict[predictions[-1]])
#   softs = softmax(preds[i])
#   rank = rank_simple(softs)

#   if labels[-1] == predictions[-1]:
#     is_equal.append(1)
#   else:
#     is_equal.append(0)

#   if samer_labels[-1] == samer_predictions[-1]:
#     samer_is_equal.append(1)
#   else:
#     samer_is_equal.append(0)

#   if barec_labels[-1] == barec_predictions[-1]:
#     barec_is_equal.append(1)
#   else:
#     barec_is_equal.append(0)

#   if abs(labels[-1] - predictions[-1]) <= 1:
#     within_one.append(1)
#   else:
#     within_one.append(0)

#   if labels[-1] in rank[:2]:
#     within_top_2.append(1)
#   else:
#     within_top_2.append(0)

#   if labels[-1] in rank[:3]:
#     within_top_3.append(1)
#   else:
#     within_top_3.append(0)

#   if labels[-1] in rank[:4]:
#     within_top_4.append(1)
#   else:
#     within_top_4.append(0)

#   rank_of_correct.append(rank.index(labels[-1])+1)
#   diff.append(max([labels[-1],predictions[-1]])-min([labels[-1],predictions[-1]]))
#   for j in range(19):
#     probs[j+1].append(softs[j])
#     ranks[j+1].append(rank[j])



# QWK = cohen_kappa_score(labels, predictions, weights='quadratic')
# acc = sum(is_equal)/len(is_equal)
# acc_samer = sum(samer_is_equal)/len(samer_is_equal)
# acc_barec = sum(barec_is_equal)/len(barec_is_equal)
# acc_within_one_level = sum(within_one)/len(within_one)
# acc_top_2 = sum(within_top_2)/len(within_top_2)
# acc_top_3 = sum(within_top_3)/len(within_top_3)
# acc_top_4 = sum(within_top_4)/len(within_top_4)
# avg_rank = sum(rank_of_correct)/len(rank_of_correct)
# avg_distance = sum(diff)/len(diff)

# print(f"Accuracy: {acc*100:.4f}")
# print(f"Accuracy with margin of one level: {acc_within_one_level*100:.4f}")
# print(f"Accuracy of top 2 choices: {acc_top_2*100:.4f}")
# print(f"Accuracy of top 3 choices: {acc_top_3*100:.4f}")
# print(f"Accuracy of top 4 choices: {acc_top_4*100:.4f}")
# print(f"Accuracy of SAMER levels: {acc_samer*100:.4f}")
# print(f"Accuracy of BAREC groups: {acc_barec*100:.4f}")
# print(f"Quadratic Weighted Kappa: {QWK*100:.4f}")
# print(f"Average rank of correct label: {avg_rank:.4f}")
# print(f"Average distance between labels and predictions: {avg_distance:.4f}")


v = {
    #'original text': original_texts,
    'text': texts,
    'label': labels,
    'prediction': predictions,
    # 'is_equal': is_equal,
    # 'samer_label': samer_labels,
    # 'samer_prediction': samer_predictions,
    # 'samer_is_equal': samer_is_equal,
    # 'barec_group_label': barec_labels,
    # 'barec_group_prediction': barec_predictions,
    # 'barec_group_is_equal': barec_is_equal,
    # 'within_one_level': within_one,
    # 'within_top2_ranks': within_top_2,
    # 'within_top3_ranks': within_top_3,
    # 'within_top4_ranks': within_top_4,
    # 'rank_of_correct_label': rank_of_correct,
    # 'diff': diff
}

# for i in range(1,20):
#   s = 'p'+str(i)
#   v[s] = probs[i]
# for i in range(1,20):
#   s = 'rank'+str(i)
#   v[s] = ranks[i]

final_df = pd.DataFrame.from_dict(v)
final_df.to_excel(out_xlsx ,index=False)

events.out.tfevents.1742932110.69641326d9f4.1960.0:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

              precision    recall  f1-score   support

           0     0.3333    0.2903    0.3103        31
           1     0.2623    0.3902    0.3137        41
           2     0.3797    0.6068    0.4671       117
           3     0.4638    0.5079    0.4848        63
           4     0.4198    0.4857    0.4503       280
           5     0.3738    0.4255    0.3980       188
           6     0.5088    0.6111    0.5553       666
           7     0.6428    0.6297    0.6362       740
           8     0.5597    0.5211    0.5397       261
           9     0.6547    0.7482    0.6983      1219
          10     0.4633    0.3033    0.3666       521
          11     0.8578    0.8061    0.8312      4266

    accuracy                         0.6984      8393
   macro avg     0.4933    0.5272    0.5043      8393
weighted avg     0.7080    0.6984    0.7004      8393

[[   9   10    8    0    1    0    1    0    0    1    0    1]
 [   8   16    9    0    1    2    1    0    1    1    0    2]
 [   2 

Binary -> 11 model , 12 model

In [ ]:
import pandas as pd

In [ ]:
# open the binary file , merge on sentence with the whole data set.
# open /content/drive/MyDrive/MBZ/Thesis/subsets_binary/2levels_8753.xlsx
# binary = pd.read_excel('/content/drive/MyDrive/MBZ/Thesis/subsets_binary/2levels_13130.xlsx')

full_data = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv')
full_data= full_data[full_data['Split'] == 'Test']
# rename word_sents in full data to text
full_data.rename(columns={'word_sents': 'text'}, inplace=True)

# merge on sentence
# full_data = full_data.merge(binary, on='text', how='left')
# create 2 data sets one for 1 and one for 2
# full_data_1 = full_data[full_data['prediction'] == 1.0]
# full_data_2 = full_data[full_data['prediction'] == 2.0]

# predict 1, with the 11s model
model11 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/11levels_26977')

# predict 2, witht the 12s model
model12 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/12levels_25544')

model19 = AutoModelForSequenceClassification.from_pretrained('Noorrabie/19levels_52521')



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

In [ ]:
# predict using the models
tokenizer11 = AutoTokenizer.from_pretrained('Noorrabie/11levels_26977')

tokenizer12 = AutoTokenizer.from_pretrained('Noorrabie/12levels_25544')

tokenizer19 = AutoTokenizer.from_pretrained('Noorrabie/19levels_52521')



tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer
from datasets import Dataset

# Batch prediction function
def batch_predict(df, model, tokenizer):
    dataset = Dataset.from_pandas(df[['text']])
    dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    trainer = Trainer(model=model, tokenizer=tokenizer)
    outputs = trainer.predict(dataset)
    preds = outputs.predictions.argmax(axis=1)
    return preds


In [ ]:
data_11= full_data[full_data['RL_num_19']<12]

In [ ]:
data_12 = full_data[full_data['RL_num_19']>=12]

In [ ]:
len(data_11)

3295

In [ ]:
len(preds_19)

7268

In [ ]:

preds_11 = batch_predict(data_11, model11, tokenizer11)
preds_11 = preds_11 + 1
data_11["final_prediction"] = preds_11

# Predict for full_data_2 using 12-class model
preds_12 = batch_predict(data_12, model12, tokenizer12)
preds_12 = preds_12 + 1
data_12["final_prediction"] = preds_12

preds_19 = batch_predict(full_data, model19, tokenizer19)
preds_19 = preds_19 + 1
full_data["final_prediction"] = preds_19

Map:   0%|          | 0/3295 [00:00<?, ? examples/s]

<ipython-input-16-62fc23538e78>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

...

Map:   0%|          | 0/3973 [00:00<?, ? examples/s]

<ipython-input-16-62fc23538e78>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


<ipython-input-23-554e20a4d068>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_12["final_prediction"] = preds_12


Map:   0%|          | 0/7268 [00:00<?, ? examples/s]

<ipython-input-16-62fc23538e78>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


In [ ]:
data_11.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/11_test_predictions.csv", index=False)
data_12.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/12_test_predictions.csv", index=False)
full_data.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets/subsets_11/19_test_predictions.csv", index=False)

In [ ]:
# get classification report for full_data_1
from sklearn.metrics import classification_report
print(classification_report(full_data_1['RL_num_19'], full_data_1['final_prediction']))

              precision    recall  f1-score   support

           1       0.33      0.73      0.45        30
           2       0.20      0.41      0.27        39
           3       0.41      0.60      0.49       116
           4       0.38      0.53      0.44        62
           5       0.48      0.46      0.47       273
           6       0.39      0.52      0.45       180
           7       0.49      0.69      0.57       635
           8       0.60      0.74      0.66       674
           9       0.71      0.49      0.58       235
          10       0.68      0.84      0.75      1009
          11       0.36      0.55      0.43       290
          12       0.00      0.00      0.00       719
          13       0.00      0.00      0.00       104
          14       0.00      0.00      0.00        83
          15       0.00      0.00      0.00        23
          16       0.00      0.00      0.00         5
          18       0.00      0.00      0.00         1

    accuracy              

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# get classification report for full_data_2
print(classification_report(full_data_2['RL_num_19'], full_data_2['final_prediction']))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         7
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00        31
           8       0.00      0.00      0.00        66
           9       0.00      0.00      0.00        26
          10       0.00      0.00      0.00       210
          11       0.00      0.00      0.00       231
          12       0.45      0.81      0.58      1226
          13       0.52      0.41      0.46       382
          14       0.59      0.55      0.57      1220
          15       0.27      0.20      0.23       279
          16       0.30      0.05      0.08       151
          17       0.00      0.00      0.00        48
          18       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Combine both dataframes
final_predictions_df = pd.concat([full_data_1, full_data_2])

# sort by ID
final_predictions_df = final_predictions_df.sort_values(by='ID')

# Save or analyze
final_predictions_df.to_csv("/content/drive/MyDrive/MBZ/Thesis/subsets_11/final_predictions.csv", index=False)


In [ ]:
print(classification_report(final_predictions_df['RL_num_19'], final_predictions_df['final_prediction']))


              precision    recall  f1-score   support

           1       0.33      0.71      0.45        31
           2       0.20      0.39      0.26        41
           3       0.41      0.60      0.48       117
           4       0.38      0.52      0.44        63
           5       0.48      0.45      0.46       280
           6       0.39      0.50      0.44       188
           7       0.49      0.65      0.56       666
           8       0.60      0.67      0.64       740
           9       0.71      0.44      0.54       261
          10       0.68      0.69      0.69      1219
          11       0.36      0.31      0.33       521
          12       0.45      0.51      0.48      1945
          13       0.52      0.33      0.40       486
          14       0.59      0.52      0.55      1303
          15       0.27      0.19      0.22       302
          16       0.30      0.04      0.08       156
          17       0.00      0.00      0.00        48
          18       0.00    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
final_predictions_df

,ID,Sentence,Word Count,Readability Level,RL_num_19,Annotator,annotation file,Project Phase,Source,Book,...,RL_num_7,RL_num_5,RL_num_3,d3lex_sents,lex_sents,d3tok_sents,text,label,prediction,final_prediction
0,1,بدأت باخباري أشياء عن ولدي أنه كسول وأنه غير مؤدب,10,10-ya,10,Zeina,Zeina_phase1_23.xlsx_revised,Phase1,Subtitles,Subtitles,...,4,2,1,بدأ ب+ خبر +ي شيء عن ولد +ي أن +ه كسول و+ أن +...,بدأ خبر شيء عن ولد أن كسول أن غير مؤدب,بدأت ب+ أخبار +ي أشياء عن ولد +ي أن +ه كسول و+...,بدأت باخباري أشياء عن ولدي أنه كسول وأنه غير مؤدب,1,1,10
1,3,فَإِلَى شَرْقِ الْبَحْرِ يَقْبَعُ مَمَرُّ الذَ...,8,30-lam,12,Rita,Rita_phase1_11.xlsx_revised,Phase1,Hindawi,Chinese Stories for Children,...,5,3,2,ف+ إلى شرق ال+ بحر قبع ممر ال+ ذهب و+ ال+ فضة .,إلى شرق بحر قبع ممر ذهب فضة .,ف+ إلى شرق ال+ بحر يقبع ممر ال+ ذهب و+ ال+ فضة .,فإلى شرق البحر يقبع ممر الذهب والفضة .,2,2,12
2,6,"إسمعي ، إكتشفت مشاكل شديدة حول "" ريك "" .",10,7-zay,7,Zeina,Zeina_phase1_15.xlsx_revised,Phase1,Subtitles,Subtitles,...,2,1,1,"اسمعي , ٱكتشف مشكلة شديد حول "" ريك "" .","اسمعي , ٱكتشف مشكلة شديد حول "" ريك "" .","اسمعي , اكتشفت مشاكل شديدة حول "" ريك "" .","إسمعي , إكتشفت مشاكل شديدة حول "" ريك "" .",1,1,7
3,9,هذا كان مثل أي شخص آخر,6,10-ya,10,Rita,Rita_phase1_1.xlsx_revised,Phase1,Subtitles,Subtitles,...,4,2,1,هذا كان مثل أي شخص آخر,هذا كان مثل أي شخص آخر,هذا كان مثل أي شخص آخر,هذا كان مثل أي شخص آخر,1,1,10
4,16,وَكَانَ يَخْلُو بِغَارِ حِرَاءٍ فَيَتَحَنَّثُ ...,22,30-lam,12,Sawsan,Sawsan_phase1_17.xlsx_revised,Phase1,Hadith,Sahih Bukhari,...,5,3,2,و+ كان خلا ب+ غار حراء ف+ تحنث في +ه و+ هو ال+...,كان خلا غار حراء تحنث في هو تعبد ليلة ذات عدد ...,و+ كان يخلو ب+ غار حراء ف+ يتحنث في +ه و+ هو ا...,وكان يخلو بغار حراء فيتحنث فيه وهو التعبد اللي...,2,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8390,68173,نعم ، هذا ما قلته,5,10-ya,10,IAA,IAA_phase1_16.xlsx_revised,Phase1,Subtitles,Subtitles,...,4,2,1,"نعم , هذا ما قال +ه","نعم , هذا ما قال","نعم , هذا ما قلت +ه","نعم , هذا ما قلته",1,1,6
8391,68175,قَالاَ حَدَّثَنَا إِبْرَاهِيمُ بْنُ سَعْدٍ ،,6,7-zay,7,IAA,IAA_phase1_16.xlsx_revised,Phase1,Hadith,Sahih Bukhari,...,2,1,1,"قال حدث +نا إبراهيم بن سعد ,","قال حدث إبراهيم بن سعد ,","قالا حدث +نا إبراهيم بن سعد ,","قالا حدثنا إبراهيم بن سعد ,",1,1,6
8392,68177,تُساورها هَفواتُ النسيم,3,30-lam,12,IAA,IAA_phase1_16.xlsx_revised,Phase1,al-Kashkuul,al-Kashkuul,...,5,3,2,ساور +ها هفوة ال+ نسيم,ساور هفوة نسيم,تساور +ها هفوات ال+ نسيم,تساورها هفوات النسيم,2,2,15
8393,68179,أَتَحَمَّلُ الْمَسْؤُولِيَّةَ، وَأُحِبُّ وَطَني.,6,6-waw,6,IAA,IAA_phase1_16.xlsx_revised,Phase1,Emarati Curriculum,Grade 1 Islamic Studies,...,2,1,1,"تحمل ال+ مسؤولية , و+ أحب وطن +ي .","تحمل مسؤولية , أحب وطن .","أتحمل ال+ مسؤولية , و+ أحب وطن +ي .","أتحمل المسؤولية , وأحب وطني .",1,1,5


In [ ]:
# check for duplicate ids in final_predictions_df
len(final_predictions_df[final_predictions_df.duplicated(subset='ID', keep=False)])

4

In [ ]:
final_predictions_df[final_predictions_df.duplicated(subset='ID', keep=False)]

,ID,Sentence,Word Count,Readability Level,RL_num_19,Annotator,annotation file,Project Phase,Source,Book,...,RL_num_7,RL_num_5,RL_num_3,d3lex_sents,lex_sents,d3tok_sents,text,label,prediction,final_prediction
4187,37137,حرية إبداء الرأي قولاً وكتابة وحرية الطباعة وح...,18,50-nun,14,IAA,IAA_phase2_MCWC_2.xlsx,Phase2,Constitutions,Lebanon's Constitution,...,6,4,3,حرية إبداء ال+ رأي قول و+ كتابة و+ حرية ال+ طب...,حرية إبداء رأي قول كتابة حرية طباعة حرية ٱجتما...,حرية إبداء ال+ رأي قولا و+ كتابة و+ حرية ال+ ط...,حرية إبداء الرأي قولا وكتابة وحرية الطباعة وحر...,2,2,14
4188,37137,حرية إبداء الرأي قولاً وكتابة وحرية الطباعة وح...,18,50-nun,14,IAA,IAA_phase2_MCWC_2.xlsx,Phase2,Constitutions,Lebanon's Constitution,...,6,4,3,حرية إبداء ال+ رأي قول و+ كتابة و+ حرية ال+ طب...,حرية إبداء رأي قول كتابة حرية طباعة حرية ٱجتما...,حرية إبداء ال+ رأي قولا و+ كتابة و+ حرية ال+ ط...,حرية إبداء الرأي قولا وكتابة وحرية الطباعة وحر...,2,2,14
6433,58949,حریة إبداء الرأي قولا وكتابة وحریة الطباعة وحر...,18,50-nun,14,Sawsan,Sawsan_phase2_1M_15.xlsx,Phase2,ReadMe++,#,...,6,4,3,حرية إبداء ال+ رأي قول و+ كتابة و+ حرية ال+ طب...,حرية إبداء رأي قول كتابة حرية طباعة حرية ٱجتما...,حرية إبداء ال+ رأي قولا و+ كتابة و+ حرية ال+ ط...,حرية إبداء الرأي قولا وكتابة وحرية الطباعة وحر...,2,2,14
6434,58949,حریة إبداء الرأي قولا وكتابة وحریة الطباعة وحر...,18,50-nun,14,Sawsan,Sawsan_phase2_1M_15.xlsx,Phase2,ReadMe++,#,...,6,4,3,حرية إبداء ال+ رأي قول و+ كتابة و+ حرية ال+ طب...,حرية إبداء رأي قول كتابة حرية طباعة حرية ٱجتما...,حرية إبداء ال+ رأي قولا و+ كتابة و+ حرية ال+ ط...,حرية إبداء الرأي قولا وكتابة وحرية الطباعة وحر...,2,2,14


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import Trainer
from transformers import Trainer
from datasets import Dataset
import pandas as pd

full_data = pd.read_csv('/content/drive/MyDrive/MBZ/Thesis/All_data_1M_morph_clean.csv')
full_data= full_data[full_data['Split'] == 'Test']
# rename word_sents in full data to text
full_data.rename(columns={'word_sents': 'text'}, inplace=True)

# Load tokenizer and model
model_name = "Noorrabie/bert_content"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Batch prediction function
def batch_predict(df, model, tokenizer):
    dataset = Dataset.from_pandas(df[['text']])
    dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

    trainer = Trainer(model=model, tokenizer=tokenizer)
    outputs = trainer.predict(dataset)
    preds = outputs.predictions.argmax(axis=1)
    return preds

preds= batch_predict(full_data, model, tokenizer)
full_data["content_level"] = preds


Map:   0%|          | 0/7268 [00:00<?, ? examples/s]

<ipython-input-15-cdf775628649>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nour-rabih (noor-rabie) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
full_data

,ID,Sentence,Word Count,Readability Level,RL_num_19,Annotator,annotation file,Project Phase,Source,Book,...,word count unique,max word count in RL,RL_num_7,RL_num_5,RL_num_3,d3lex_sents,lex_sents,d3tok_sents,text,final_prediction
9,10,خلطتها وخلطتها وخلطتها إلى أن صارت عجينة طرية.,9,6-waw,6,Rita,Rita_phase1_14.xlsx_revised,Phase1,Mama Makes Bread,Mama Makes Bread,...,7,9,2,1,1,خلط +ها و+ خلط +ها و+ خلط +ها إلى أن صار عجينة...,خلط خلط خلط إلى أن صار عجينة طري .,خلطت +ها و+ خلطت +ها و+ خلطت +ها إلى أن صارت ع...,خلطتها وخلطتها وخلطتها إلى أن صارت عجينة طرية .,4
19,20,ونال المدرب المعروف محمد إبراهيم ، القادسية ، ...,21,20-kaf,11,Rita,Rita_phase1_18.xlsx_revised,Phase1,WikiNews,Sports,...,18,20,4,2,1,"و+ نال ال+ مدرب ال+ معروف محمد إبراهيم , ال+ ق...","نال مدرب معروف محمد إبراهيم , قادسية , لقب أفض...","و+ نال ال+ مدرب ال+ معروف محمد إبراهيم , ال+ ق...","ونال المدرب المعروف محمد إبراهيم , القادسية , ...",5
20,21,وجاء زميله في الفريق جراح العتيقي كأفضل لاعب ص...,23,30-lam,12,Samar,Samar_phase1_9.xlsx_revised,Phase1,WikiNews,Sports,...,22,1000,5,3,2,و+ جاء زميل +ه في ال+ فريق جراح ال+ عتيقي ك+ أ...,جاء زميل في فريق جراح عتيقي أفضل لاعب صاحب روح...,و+ جاء زميل +ه في ال+ فريق جراح ال+ عتيقي ك+ أ...,وجاء زميله في الفريق جراح العتيقي كأفضل لاعب ص...,5
27,28,أعلنت شركة كاسبرسكي لاب المختصة بأمن الحواسيب ...,21,50-nun,14,Zeina,Zeina_phase1_14.xlsx_revised,Phase1,WikiNews,Science,...,19,1000,6,4,3,أعلن شركة كاسبرسكي لاب ال+ مختص ب+ أمن ال+ حاس...,أعلن شركة كاسبرسكي لاب مختص أمن حاسوب أمس ٱثني...,أعلنت شركة كاسبرسكي لاب ال+ مختصة ب+ أمن ال+ ح...,أعلنت شركة كاسبرسكي لاب المختصة بأمن الحواسيب ...,6
28,29,قابَلْتُكَ فِي الْفُنْدُقِ، عَرَفْتُ اسْمَكَ.,7,6-waw,6,Rita,Rita_phase1_15.xlsx_revised,Phase1,Hindawi,The Merchant Marmar,...,5,9,2,1,1,"قابل +ك في ال+ فندق , عرف ٱسم +ك .","قابل في فندق , عرف ٱسم .","قابلت +ك في ال+ فندق , عرفت اسم +ك .","قابلتك في الفندق , عرفت اسمك .",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68173,68174,في منتصف القرن العشرين، بدأ عدد قليل من العلما...,35,50-nun,14,IAA,IAA_phase1_16.xlsx_revised,Phase1,Wikipedia,Artificial Intelligence,...,28,1000,6,4,3,"في منتصف ال+ قرن ال+ عشرون , بدأ عدد قليل من ا...","في منتصف قرن عشرون , بدأ عدد قليل من عالم ٱستك...","في منتصف ال+ قرن ال+ عشرين , بدأ عدد قليل من ا...","في منتصف القرن العشرين , بدأ عدد قليل من العلم...",6
68175,68176,وشاعرٌ لا تستحي أن تصـفعه,5,8-Ha,8,IAA,IAA_phase1_16.xlsx_revised,Phase1,al-Kashkuul,al-Kashkuul,...,5,11,3,2,1,و+ شاعر لا ٱستحيى أن صفع +ه,شاعر لا ٱستحيى أن صفع,و+ شاعر لا تستحي أن تصفع +ه,وشاعر لا تستحي أن تصفعه,3
68177,68178,(\t) وُجوبُ طَلَبِ تَزْكِيَةِ النَّفْسِ بِالإي...,17,50-nun,14,IAA,IAA_phase1_16.xlsx_revised,Phase1,Emarati Curriculum,Grade 7 Arabic Language,...,11,1000,6,4,3,( ) وجوب طلب تزكية ال+ نفس ب+ ال+ إيمان و+ ال+...,"( ) وجوب طلب تزكية نفس إيمان عمل صالح , تهذيب ...",( ) وجوب طلب تزكية ال+ نفس ب+ ال+ إيمان و+ ال+...,( ) وجوب طلب تزكية النفس بالإيمان والعمل الصال...,5
68179,68180,إِنَّهُ ابْني الوَحيدُ،,4,10-ya,10,IAA,IAA_phase1_16.xlsx_revised,Phase1,Emarati Curriculum,Grade 3 Islamic Studies,...,3,15,4,2,1,"إن +ه ٱبن +ي ال+ وحيد ,","إن ٱبن وحيد ,","إن +ه ابن +ي ال+ وحيد ,","إنه ابني الوحيد ,",4
